In [ ]:
import pandas as pd
import numpy as np

import sys
import os
username = os.environ.get('USER')
sys.path.append(f'/data/workspace/{username}')

Run the following code in Terminal from the project root dir:

```bash
python scripts/build_original_df.py --dataset sample-small
```

# Feature Engineering

In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.abspath("../../"), "src"))
from extract_header_features import *
from extract_text_features import *
from extract_url_features import *


In [ ]:
# from git_repo.src.extract_header_features import (
#     has_dmarc_authentication, get_dkim_result,
#     get_spf_result, get_dmarc_result,
#     dkim_domain_matches_sender, has_attachment,
#     number_of_received, to_from_match,
#     spf_email_matches_sender
# )

# from git_repo.src.extract_text_features import (
#     non_ascii_present, hidden_text_present, html_parsing_error,
#     word_count, readable_proportion, whitespace_ratio,
#     alphabet_proportion, check_grammar, english_french_proportion
# )

# features_df = pd.DataFrame({
#     "dmarc_authentication_present": has_dmarc_authentication(original_df['Authentication-Results']),
#     "dkim_result": get_dkim_result(original_df['Authentication-Results']),
#     "spf_result": get_spf_result(original_df['received-spf']),
#     "dmarc_result": get_dmarc_result(original_df['Authentication-Results']),
#     "dkim_sender_domains_match": dkim_domain_matches_sender(
#         original_df['DKIM-Signature'],
#         original_df['From_email_domain']
#     ),
#     "attachments_present": has_attachment(original_df['attachment_types']),
#     "routing_length": number_of_received(original_df['Received']),
#     "to_from_addresses_match": to_from_match(original_df['From_email'], original_df['To_email']),
#     "sender_email_spf_match": spf_email_matches_sender(
#         original_df['received-spf'], original_df['From_email']
#     ),

#     "non_ascii_present": non_ascii_present(original_df['text_clean']),
#     "hidden_text_present": hidden_text_present(original_df['text_html']),
#     "html_parsing_error": html_parsing_error(original_df['text_html']),
#     "word_count": word_count(original_df['text_clean']),
#     "readable_proportion": readable_proportion(original_df['text_clean'], original_df['text_html']),
#     "whitespace_ratio": whitespace_ratio(original_df['text_plain']),
#     "alphabet_proportion": alphabet_proportion(original_df['text_clean']),
#     "grammar_error_rate": check_grammar(original_df['text_plain'], original_df['Content-Language']),
#     "english_french_proportion": english_french_proportion(original_df['text_plain']),

    
# })

# features_df

In [ ]:
# from git_repo.src.extract_url_features import (
#     get_url_count, has_accessible_url, has_redirected_url,
#     has_ip_url, has_http_only, has_at_symbol,
#     has_port_number, has_long_url, has_multiple_subdomains
# )


# features_df_2 = pd.DataFrame({
#     "url_count": get_url_count(original_df['urls']),
#     #"all_urls_accessible": has_accessible_url(original_df['urls']),
#     #"urls_redirected": has_redirected_url(original_df['urls']),
#     "ip_addr_urls": has_ip_url(original_df['urls']),
#     "http_urls_present": has_http_only(original_df['urls']),
#     "url_at_symbol": has_at_symbol(original_df['urls']),
#     "url_port_number": has_port_number(original_df['urls']),
#     "any_long_urls": has_long_url(original_df['urls']),
#     "url_multiple_subdomains": has_multiple_subdomains(original_df['urls'])
# })

# features_df_2

# Read in data [start from here]

In [ ]:
original_df = pd.read_parquet('/data/workspace/jiaquan/2025-cv/data/sampled-dataset/raw/sample-large.parquet')
original_df

In [ ]:
# Full dataset
original_df = pd.read_parquet('/data/workspace/dataset/full-dataset/raw/train.parquet')
original_df

In [ ]:
input_df = pd.read_parquet('/data/workspace/jiaquan/2025-cv/data/sampled-dataset/processed/sample-large.parquet')
# input_df

In [ ]:
# Full dataset
input_df = pd.read_parquet('/data/workspace/dataset/full-dataset/processed/train.parquet')
input_df

In [ ]:
# Join original_df with features_df and features_df_2
combined_df = original_df.join(input_df)
combined_df

In [ ]:
# Train test split the combined_df
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(combined_df, test_size=0.3, random_state=42)

In [ ]:
X_train = train_df.drop(columns=['target_1'])
y_train = train_df['target_1']

X_test = test_df.drop(columns=['target_1'])
y_test = test_df['target_1']


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_num = label_encoder.fit_transform(y_train)
y_test_num = label_encoder.transform(y_test)

# Preprocessing

In [ ]:
numeric_feats = [
    "routing_length", "html_parsing_error", "word_count", 
    "readable_proportion", "whitespace_ratio", "alphabet_proportion",
    "grammar_error_rate", "english_french_proportion",
    "url_count"
]  # apply scaling

binary_feats = [
    "is_multipart",
    "dmarc_authentication_present", "dkim_result",
    "spf_result", "dmarc_result", "dkim_sender_domains_match",
    "attachments_present", "to_from_addresses_match", "sender_email_spf_match",
    "non_ascii_present", "hidden_text_present"
    #"ip_addr_urls", "http_urls_present", "url_at_symbol",
    #"url_port_number", "any_long_urls", "url_multiple_subdomains"
    ]

text_feats = [
    "Content_types" ,
    "urls",
    "attachment_types",
    "Subject",
    "text_preprocessed",
    ]

categorical_feats = [
    "From_name", "From_email", "From_email_domain", 
    "To_name", "To_email", "To_email_domain",
    "Content-Language"
    ]  # apply one-hot encoding

passthrough_feats = [""]  # do not apply any transformation

drop_feats = [
    "From",                         # Info extracted to From_name, From_email, From_email_domain
    "To",                           # Info extracted to To_name, To_email, To_email_domain
    "Received",                     # Info extracted to routing_length
    "Authentication-Results",       # Info extracted to dmarc_authentication_present, dkim_result, spf_result, dmarc_result
    "received-spf",                 # Info extracted to spf_result, sender_email_spf_match
    "DKIM-Signature",               # Info extracted to dkim_sender_domains_match
    "Reply-To",                     # Mostly missing, not useful
    "Return-Path",                  # Mostly missing, not useful
    "text_plain",                   
    "text_clean", 
    "text_html", 
    # "attachment_types",             # Info extracted to attachments_present
    # "urls",                         # Info extracted to url_count, http_urls_present, ip_addr_urls, url_at_symbol, url_port_number, any_long_urls, url_multiple_subdomains (not used yet)
    "target_2",                     # Level 2 target variable
    "target_3",                     # Level 3 target variable
    # "Subject",                      # To be used later
    # "text_preprocessed",            # To be used later
    # "Content_types"                 # To be used later
]



# EDA

In [ ]:
eda_df = X_train.copy()
# subset only the numerical features
num_df = eda_df[numeric_feats]


# Perform correlation analysis
correlation_matrix = num_df.corr()

# Plot the correlation matrix
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
# Filter out the drop features
eda_df = eda_df.drop(columns=drop_feats)

In [ ]:
eda_df

In [ ]:
# Check for missing values proportions
eda_df.isnull().sum()/len(eda_df) * 100

## Column transformer

In [ ]:
for col in X_train.columns:
    if X_train[col].apply(lambda x: isinstance(x, (list, np.ndarray))).any():
        print(col)

In [ ]:
cols = ["Content_types", "attachment_types", "urls"]

for col in cols:
    X_train[col] = X_train[col].apply(lambda x: " ".join(x) if isinstance(x, (list, np.ndarray)) else str(x))
    X_test[col] = X_test[col].apply(lambda x: " ".join(x) if isinstance(x, (list, np.ndarray)) else str(x))


In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)

from sklearn.preprocessing import FunctionTransformer
flatten = FunctionTransformer(lambda x: x.ravel(), validate=False),



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import make_column_transformer

numeric_transformer = make_pipeline(StandardScaler())

binary_transformer = make_pipeline(OneHotEncoder(handle_unknown='ignore', drop='if_binary'))

text_transformer = make_pipeline(CountVectorizer())

categorical_transformer = make_pipeline(SimpleImputer(strategy="constant", fill_value="None"), OneHotEncoder(handle_unknown='ignore'))


In [ ]:
X_train

In [ ]:
X_train["Subject"] = X_train["Subject"].fillna("")
X_train["text_preprocessed"] = X_train["text_preprocessed"].fillna("")
X_test["Subject"] = X_test["Subject"].fillna("")
X_test["text_preprocessed"] = X_test["text_preprocessed"].fillna("")
X_train

In [ ]:
# preprocessor = make_column_transformer(
#     (numeric_transformer, numeric_feats),
#     (binary_transformer, binary_feats),
#     (text_transformer, text_feats),
#     (categorical_transformer, categorical_feats),
#     ("drop", drop_feats)
# )

tfidf_subject_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="")),
    ("flatten", FunctionTransformer(lambda x: x.ravel(), validate=False)),
    ("tfidf", TfidfVectorizer())
])

tfidf_text_preprocessed_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="")),
    ("flatten", FunctionTransformer(lambda x: x.ravel(), validate=False)),
    ("tfidf", TfidfVectorizer())
])

preprocessor = make_column_transformer(
    (numeric_transformer, numeric_feats),
    (binary_transformer, binary_feats),
    (categorical_transformer, categorical_feats),
    (CountVectorizer(), "Content_types"),
    (CountVectorizer(), "urls"),
    (CountVectorizer(), "attachment_types"),
    (tfidf_subject_pipeline, ["Subject"]),
    (tfidf_text_preprocessed_pipeline, ["text_preprocessed"]),
    ("drop", drop_feats)
)

In [ ]:
preprocessor

In [ ]:
pipe_rf_demo = make_pipeline(
    preprocessor, RandomForestClassifier(max_depth=2, n_estimators=3, random_state=123)
)
pipe_rf_demo.fit(X_train, y_train_num)


In [ ]:
# Code adapted from DSCI571: Lecture 4 
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores.iloc[i], std_scores.iloc[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [ ]:
scoring = ['accuracy', 'f1', 'precision', 'recall']
results_dict = {}
results_dict["rf"] = mean_std_cross_val_scores(pipe_rf_demo, X_train, y_train_num, scoring=scoring, return_train_score=True)
pd.DataFrame(results_dict)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = pipe_rf_demo.predict(X_train)
cm = confusion_matrix(y_train_num, y_pred)

TN, FP, FN, TP = cm.ravel()
fpr = FP / (FP + TN)
print("False Positive Rate:", fpr)

# Hyperparameter optimization

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, lognorm, loguniform, randint, uniform, norm, randint


In [ ]:
param_dist = {
    "randomforestclassifier__criterion": ["gini", "entropy", "log_loss"],
    "randomforestclassifier__n_estimators": randint(10, 200),
    "randomforestclassifier__max_depth": randint(1, 20),
    "randomforestclassifier__min_samples_split": randint(2, 20),
    "randomforestclassifier__min_samples_leaf": randint(1, 20),
    "randomforestclassifier__max_features": uniform(0.1, 0.9),
    "randomforestclassifier__bootstrap": [True, False],
    "columntransformer__pipeline-4__tfidf__max_features": [None, 1000, 5000, 10000, 15000, 20000],
    "columntransformer__pipeline-5__tfidf__max_features": [None, 1000, 5000, 10000, 15000, 20000]
}

pipe_rf = make_pipeline(preprocessor, RandomForestClassifier(random_state=123, class_weight="balanced", n_jobs=-1))

random_search_rf = RandomizedSearchCV(
    pipe_rf,
    param_distributions=param_dist,
    n_iter=10,
    scoring="f1",
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=123,
)

In [ ]:
random_search_rf.fit(X_train, y_train_num)

In [ ]:
random_search_rf.best_params_

In [ ]:
scoring = ['accuracy', 'f1', 'precision', 'recall']
results_dict = {}
results_dict["rf"] = mean_std_cross_val_scores(random_search_rf.best_estimator_, X_train, y_train_num, scoring=scoring, return_train_score=True)
pd.DataFrame(results_dict)

In [ ]:
y_pred = random_search_rf.predict(X_train)
cm = confusion_matrix(y_train_num, y_pred)

TN, FP, FN, TP = cm.ravel()
fpr = FP / (FP + TN)
print("False Positive Rate:", fpr)

In [ ]:
import pickle

In [ ]:
# Save the parameters as pickle object
rf_param = random_search_rf.best_params_

pickle.dump(rf_param, open("rf_param.pkl", "wb"))


In [ ]:
param = pickle.load(open("rf_param.pkl", "rb"))
param

In [ ]:
pipe_rf = make_pipeline(preprocessor, RandomForestClassifier(random_state=123, class_weight="balanced", n_jobs=-1,
    n_estimators=param["randomforestclassifier__n_estimators"],
    max_depth=param["randomforestclassifier__max_depth"],
    min_samples_split=param["randomforestclassifier__min_samples_split"],
    min_samples_leaf=param["randomforestclassifier__min_samples_leaf"],
    max_features=param["randomforestclassifier__max_features"],
    bootstrap=param["randomforestclassifier__bootstrap"]
))

pipe_rf.set_params(**{
    "columntransformer__pipeline-4__tfidf__max_features": param["columntransformer__pipeline-4__tfidf__max_features"],
    "columntransformer__pipeline-5__tfidf__max_features": param["columntransformer__pipeline-5__tfidf__max_features"]
})

In [ ]:
pipe_rf.fit(X_train, y_train_num)

In [ ]:
scoring = ['accuracy', 'f1', 'precision', 'recall']
results_dict = {}
results_dict["rf"] = mean_std_cross_val_scores(pipe_rf, X_train, y_train_num, scoring=scoring, return_train_score=True)
pd.DataFrame(results_dict)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = pipe_rf.predict(X_train)
cm = confusion_matrix(y_train_num, y_pred)

TN, FP, FN, TP = cm.ravel()
fpr = FP / (FP + TN)
print("False Positive Rate:", fpr)


In [ ]:
cm

# X_test

In [ ]:
X_test

In [ ]:
X_test_transformed = preprocessor.transform(X_test)

In [ ]:
# Test the model on the test set
y_pred_test = pipe_rf.predict(X_test)

In [ ]:
# Get the f1-score on the test set
from sklearn.metrics import f1_score
f1 = f1_score(y_test_num, y_pred_test)
print("F1 Score on Test Set:", f1)

In [ ]:
cm_test = confusion_matrix(y_test_num, y_pred_test)
TN_test, FP_test, FN_test, TP_test = cm_test.ravel()

cm_test

In [ ]:
fpr_test = FP_test / (FP_test + TN_test)
print("False Positive Rate on test set:", fpr_test)

# SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
# from collections import Counter

# counter = Counter(y_train_num)
# print('Before', counter)

# X_train_transformed = preprocessor.fit_transform(X_train)

# # oversampling the train dataset using SMOTE
# smt = SMOTE()
# X_train_sm, y_train_sm = smt.fit_resample(X_train_transformed, y_train_num)

# counter = Counter(y_train_sm)
# print('After', counter)

In [ ]:
from imblearn.pipeline import make_pipeline as make_pipeline_imb

pipe_smote = make_pipeline_imb(
    preprocessor,
    SMOTE(random_state=123),
    RandomForestClassifier(random_state=123, n_jobs=-1,) 
)

pipe_smote


In [ ]:
param_dist_smote = {
    "randomforestclassifier__criterion": ["gini", "entropy", "log_loss"],
    "randomforestclassifier__n_estimators": randint(10, 200),
    "randomforestclassifier__max_depth": randint(1, 20),
    "randomforestclassifier__min_samples_split": randint(2, 20),
    "randomforestclassifier__min_samples_leaf": randint(1, 20),
    "randomforestclassifier__max_features": uniform(0.1, 0.9),
    "randomforestclassifier__bootstrap": [True, False],
    "columntransformer__pipeline-4__tfidf__max_features": [None, 1000, 5000, 10000, 15000, 20000],
    "columntransformer__pipeline-4__tfidf__max_df": [0.8, 0.9, 1.0],         
    "columntransformer__pipeline-4__tfidf__min_df": [1, 3, 5],
    "columntransformer__pipeline-5__tfidf__max_features": [None, 1000, 5000, 10000, 15000, 20000],
    "columntransformer__pipeline-5__tfidf__max_df": [0.8, 0.9, 1.0],         
    "columntransformer__pipeline-5__tfidf__min_df": [1, 3, 5],
    "smote__k_neighbors": [3, 5, 7],
    "smote__sampling_strategy": ['auto', 0.8]
}


random_search_rf_smote = RandomizedSearchCV(
    pipe_smote,
    param_distributions=param_dist_smote,
    n_iter=100,
    scoring="f1",
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=123,
)

In [ ]:
random_search_rf_smote.fit(X_train, y_train_num)

In [ ]:
random_search_rf_smote.best_params_

In [ ]:
smote_best_param = random_search_rf_smote.best_params_
pipe_smote.set_params(**smote_best_param)

In [ ]:
scoring = ['accuracy', 'f1', 'precision', 'recall']
results_dict = {}
results_dict["rf"] = mean_std_cross_val_scores(pipe_smote, X_train, y_train_num, scoring=scoring, return_train_score=True)
pd.DataFrame(results_dict)



In [ ]:
pipe_smote.fit(X_train, y_train_num)

In [ ]:
y_pred = pipe_smote.predict(X_train)
cm = confusion_matrix(y_train_num, y_pred)

TN, FP, FN, TP = cm.ravel()
fpr = FP / (FP + TN)
print("False Positive Rate:", fpr)


In [ ]:
# Test the model on the test set
y_pred_test = pipe_smote.predict(X_test)


# Get the f1-score on the test set
f1 = f1_score(y_test_num, y_pred_test)
print("F1 Score on Test Set:", f1)

In [ ]:
cm_smote = confusion_matrix(y_test_num, y_pred_test)
TN_smote, FP_smote, FN_smote, TP_smote = cm_smote.ravel()
cm_smote


In [ ]:
fpr_smote = FP_smote / (FP_smote + TN_smote)
print("False Positive Rate on test set:", fpr_smote)

In [ ]:
# Save the parameters as pickle object
rf_smote_param = random_search_rf_smote.best_params_

pickle.dump(rf_smote_param, open("rf_smote_param.pkl", "wb"))
